In [109]:
# %pip install git+https://github.com/tensorflow/docs

import os.path
from typing import List

import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


# Config

In [110]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344 Version 12.13.453.3037 (Jul 11 2022/18:39:20) [PUBLIC] <Releases/12.13>
RANKED_ID = 1100    # 1090 normal game 1100 ranked game
PATCH: date = date(2022, 7, 1)  # date(2022, 7, 1)

TARGETNAME = 'placement'

In [111]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [112]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,n10cpp3NJFFDN5ER7FQDAWDzPKOhJW9rOqp1lWY_rYb2vVs,eKIt8vWw_Xjp_fBCJ24Z3TvO8cPskYOodCr4XFZ35TG5XXxFgDWpJC--,Ey9Mjy_QrBiAgOSo4cWQI8u7NYuwpEyBWQqQIP0NND7XAYoyp7lWX31AbSRxsye0DX59zO4llBZLpA,NikolaJokic2x,5026,1657639633000,38,n10cpp3NJFFDN5ER7FQDAWDzPKOhJW9rOqp1lWY_rYb2vVs,NikolaJokic2x,633,I,113,58,False,False,True,False
1,DpOFwsnNPs01F0PGzBaZXx77dRVRpmEinXM5fYwk2K9d,7oIAu4T2gSCmvfNZRzISAYiDHJjCAirWgwsBkJppm8MllMw,0K48GAFGUcLoqA8K0JAoOkFUiNNkmfy3ebLt5d24rZmpilJxwPq85c4yBwcamh7j3rnAUYM0DgRVqg,Typo Vampire,4075,1657615099000,99,DpOFwsnNPs01F0PGzBaZXx77dRVRpmEinXM5fYwk2K9d,Typo Vampire,668,I,220,149,False,False,False,False
2,zu74J2QwsNXXnU9ITZKtqzw3gbnxuFwGhNo38nZZCVIj,Q1uBipY4-_xKfrasWbtKEfkngMb0i4tDJTPNf_7zoYaYv3o,bNv1b-KBAJi6WTrnqjRDcTxnYBijLvDcIDaMTfrSGTm14jnTuSS4rS1s7aPDgawoPc-UIetKInfsJw,Kahdei,5389,1657689794514,377,zu74J2QwsNXXnU9ITZKtqzw3gbnxuFwGhNo38nZZCVIj,Kahdei,1211,I,276,134,True,False,False,False
3,H79RhtkoeHBWV_fT0OleEz570Un-6H_DKFW9-BZTGhdRA8E,Bm9sSal__SlnEBXTcFtaOSv0_2kUHdzo4YrxAaXF_CBxpfJHm0LEqBK9,BuPGnWoyn4oDOgWiLBH3aa-HZ_M2n3MCjhtdzuIR2ITfo9bEw-QWgPHozFy7h4cUOaJfX79aYh2_JQ,Dawnglade,4409,1657548556000,31,H79RhtkoeHBWV_fT0OleEz570Un-6H_DKFW9-BZTGhdRA8E,Dawnglade,1012,I,147,66,False,False,False,True
4,cAP-PDW31T8bNOhYrG6tSJLCLE3raXBcfFJyDIoZ9VNkEow,uQPBLdyPBDnQhfDMi6hYdu0PDRC_ySQw6OExLN5To7RsyLl4Y7q_ycSx,RX1PazO3ViQ9KLztmHGQ437gOvXd3ANBe9oihN8a0SJUOg1W9Q3p-xreolnL2dVcMiFTbLvdA2xGYg,Kjaos,29,1657622981000,1,cAP-PDW31T8bNOhYrG6tSJLCLE3raXBcfFJyDIoZ9VNkEow,Kjaos,1291,I,187,83,True,False,False,False
5,_Eh36dpN-BShugt-KSnUaxUcx-Sms3QkbTUpzClCBWad,MaE7nKhoBjusRnFCK3iEYkwcdkjDOpYzlv8KR9uI_Z92Zpw,fYgCqCzVJP_6RUr-_H3XOLU8sLArrWUveJ7cGtN3tjs73mXchlhd-LzV7F4qhJRm3G1c4SJP_RfQEw,Omnî,5131,1657642995000,554,_Eh36dpN-BShugt-KSnUaxUcx-Sms3QkbTUpzClCBWad,Omnî,1223,I,225,137,True,False,False,False
6,PUAUSrvm8GyU8agfT1o4Zix0QM4Mhn0tckLkZnJv3Nlv,4XegHrPOfLmw_8kpLkbdgpNhsTFbaC-hyAzYhtmh0p43hUc,s5t4w0PoHiLnwcC8k18bHMGV9tQXjZJIIO5XrYLPzRJrhb40T3vvpoeNUAKSofyBHugAC_dCR2xYRQ,Angora,746,1657632941000,258,PUAUSrvm8GyU8agfT1o4Zix0QM4Mhn0tckLkZnJv3Nlv,Angora,1232,I,190,105,True,False,False,True
7,ptR5mrtgRrrgMG1G2M1K_MsDuaQr2Jas5oNGEmDh4_DHaw,qjrD6bDUwJmnyQksJyVXssRSsYoyGG0GRW_p-fYbXSAcNzQ,yBVhO5SSTZSXnLjKV23Hvwb_YTy9JOXzyxqU-SfdBEaBma6AtDaHOQYQBianCbGxMOyXUKPz7mDgkA,oubo,9,1657457107000,256,ptR5mrtgRrrgMG1G2M1K_MsDuaQr2Jas5oNGEmDh4_DHaw,oubo,1039,I,226,145,True,False,False,False
8,fzKEcJ-8-ai-AJUNpELVB2jyX_eQkzmxJI_zUCLkT-4YMwY,8Wq9OVVYFY1oRkyz-djSbJjIwqqqX_O0JrpCncny00DT9gr1rkvo7wBf,EAYrnw4XA-TiYNybbx9Gqof90Hz0ax-qjgFeVkViL6fYTGn4R58iMXaomTBQYohaTPmpcXaWN7DMCw,Anivia Kingdom,5067,1657667122251,12,fzKEcJ-8-ai-AJUNpELVB2jyX_eQkzmxJI_zUCLkT-4YMwY,Anivia Kingdom,529,I,198,121,True,False,False,False
9,2Yjl7GqbkMho75GBU3W-ND8HR63VAc0xhB9xPK1L5g23alA,r2TBD8HMB03UJ4NX3Hkagehu8ZhmbU0FyDdpox040JAJ2cif19e1j2rQ,smP31AzH271-mYIonzAz1WCLdVAVtQzrP7q7jKRrfKojEQQTI64vG1kB5faZSOlOH1L25aGCO0HySA,Huge Hinge Hunk,16,1657632941000,301,2Yjl7GqbkMho75GBU3W-ND8HR63VAc0xhB9xPK1L5g23alA,Huge Hinge Hunk,670,I,110,57,False,False,False,True


# Load unique matches id

In [113]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [114]:
len(uniq_matches_id)

908

In [115]:
len(matches_asset)

1044

In [116]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [117]:
len(uniq_matches)

908

## Filter by LATEST_RELEASE version and RANKED_ID game.

In [118]:
matches_asset[-1]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [119]:
matches_asset[-1]['info']['queue_id']

1100

In [120]:
latest_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version']) and (
    RANKED_ID == match['info']['queue_id'])]


In [121]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [122]:
timedelta(days=3)

datetime.timedelta(days=3)

In [123]:
latest_3d_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                     and ((datetime.now() - timedelta(days=3)) <= datetime.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [124]:
len(latest_matches)

663

In [125]:
len(latest_patch_matches)

0

In [126]:
len(latest_3d_matches)

0

# Process api details to datasets rows

In [127]:
def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [128]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)
matches_3d_array = process_matches(latest_3d_matches)

In [129]:
matches_array[0]
# len(matches_array)

{'match_id': 'OC1_526778444',
 'placement': 1,
 'augment0': 'TFT6_Augment_Electrocharge1',
 'augment1': 'TFT7_Augment_ShapeshifterBeastsDen',
 'augment2': 'TFT6_Augment_ComponentGrabBag',
 'Set7_Bruiser': 0,
 'Set7_Dragonmancer': 2,
 'Set7_Jade': 1,
 'Set7_Legend': 0,
 'Set7_Mirage': 0,
 'Set7_Ragewing': 1,
 'Set7_Shapeshifter': 1,
 'Set7_Swiftshot': 1,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'TFT7_Sett': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Sett_item0': 'TitanicHydra',
 'TFT7_Karma': 2,
 'TFT7_Ashe': 2,
 'TFT7_LeeSin': 2,
 'TFT7_Swain': 2,
 'TFT7_Swain_item0': 'Quicksilver',
 'TFT7_Swain_item1': 'GuinsoosRageblade',
 'TFT7_Swain_item2': 'ArchangelsStaff',
 'TFT7_Neeko': 2,
 'TFT7_Neeko_item0': 'RedBuff',
 'TFT7_Neeko_item1': 'Morellonomicon',
 'TFT7_Neeko_item2': 'HextechGunblade',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'RapidFireCannon',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'Redemption',
 'TFT7_Yasuo': 2,
 'TFT7_Yasuo_item0': 'TitansRes

In [130]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)
matches_league_3d_df = pd.json_normalize(matches_3d_array)

In [131]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Dragonmancer,Set7_Jade,Set7_Legend,Set7_Mirage,...,TFT7_Heimerdinger_item2,TFT7_Senna_item0,TFT7_Yone_item2,TFT7_Sejuani_item2,TFT7_Skarner_item2,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,OC1_526778444,1,TFT6_Augment_Electrocharge1,TFT7_Augment_ShapeshifterBeastsDen,TFT6_Augment_ComponentGrabBag,0.0,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OC1_526778444,6,TFT7_Augment_GuardianTrait,TFT6_Augment_TargetDummies,TFT7_Augment_CannoneerEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OC1_526778444,5,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_SecondWind2,TFT7_Augment_MirageEmblem,1.0,0.0,NaN,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OC1_526778444,3,TFT6_Augment_SecondWind1,TFT6_Augment_Ascension,TFT7_Augment_UrfsGrabBag1,0.0,2.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OC1_526778444,4,TFT6_Augment_Electrocharge1,TFT6_Augment_ComponentGrabBag,TFT7_Augment_GuardianEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5299,OC1_526856179,7,TFT7_Augment_AFK,TFT7_Augment_UrfsGrabBag2,TFT6_Augment_Diversify1,2.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5300,OC1_526856179,2,TFT6_Augment_SecondWind1,TFT6_Augment_HighEndShopping,TFT6_Augment_CyberneticImplants1,1.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5301,OC1_526856179,8,TFT6_Augment_SecondWind1,TFT6_Augment_LudensEcho3,TFT6_Augment_PandorasItems,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5302,OC1_526856179,6,TFT6_Augment_Electrocharge1,TFT7_Augment_RevelEmblem2,TFT6_Augment_CelestialBlessing1,1.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bruiser,Set7_Dragonmancer,Set7_Jade,Set7_Legend,Set7_Mirage,...,TFT7_Heimerdinger_item2,TFT7_Senna_item0,TFT7_Yone_item2,TFT7_Sejuani_item2,TFT7_Skarner_item2,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,OC1_526778444,1,TFT6_Augment_Electrocharge1,TFT7_Augment_ShapeshifterBeastsDen,TFT6_Augment_ComponentGrabBag,0.0,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OC1_526778444,6,TFT7_Augment_GuardianTrait,TFT6_Augment_TargetDummies,TFT7_Augment_CannoneerEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OC1_526778444,5,TFT6_Augment_ThrillOfTheHunt1,TFT6_Augment_SecondWind2,TFT7_Augment_MirageEmblem,1.0,0.0,NaN,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OC1_526778444,3,TFT6_Augment_SecondWind1,TFT6_Augment_Ascension,TFT7_Augment_UrfsGrabBag1,0.0,2.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OC1_526778444,4,TFT6_Augment_Electrocharge1,TFT6_Augment_ComponentGrabBag,TFT7_Augment_GuardianEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5299,OC1_526856179,7,TFT7_Augment_AFK,TFT7_Augment_UrfsGrabBag2,TFT6_Augment_Diversify1,2.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5300,OC1_526856179,2,TFT6_Augment_SecondWind1,TFT6_Augment_HighEndShopping,TFT6_Augment_CyberneticImplants1,1.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5301,OC1_526856179,8,TFT6_Augment_SecondWind1,TFT6_Augment_LudensEcho3,TFT6_Augment_PandorasItems,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5302,OC1_526856179,6,TFT6_Augment_Electrocharge1,TFT7_Augment_RevelEmblem2,TFT6_Augment_CelestialBlessing1,1.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [133]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [134]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)
matches_league_3d_df = reorder_df_col(matches_league_3d_df)

# Output dataframes

In [135]:
# matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
# matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)
matches_league_3d_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.pickle'))
matches_league_3d_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.csv'), index=False)

# End